# Preprocessing

## Read Tweets/Retweets:

In [24]:
import pickle
import os, json
from os import path
import pandas as pd
import numpy as np
import glob
import timeit
start = timeit.timeit()
pd.set_option('display.max_columns', None)

#FJERN POLITIFACT 14548 - 1054731691875618829 ELLER KJØR MED ERRORS=SURROGATEESCAPE 
#GJENTA FOR REAL
tweets_per_article = []
fakeArticlesPD = pd.DataFrame()
realArticlesPD = pd.DataFrame()
fakeUsers = {}
realUsers = {}
fakeRetweets = {}
realRetweets = {}
fakeTweets = {}
realTweets = {}

for i in range(2):
    if i == 0:
        authenticity = 'fake'
        #i = 1
    else:
        #CHECKPOINT
        checkpoint = timeit.timeit()
        print(checkpoint - start)

        with open('tweets_per_article.pkl', 'wb') as f:
            pickle.dump(tweets_per_article, f)

        with open('../Visualization/fakeUsers.pickle', 'wb') as handle:
            pickle.dump(fakeUsers, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('../Visualization/realUsers.pickle', 'wb') as handle:
            pickle.dump(realUsers, handle, protocol=pickle.HIGHEST_PROTOCOL)


        with open('../Visualization/fakeRetweets.pickle', 'wb') as handle:
            pickle.dump(fakeRetweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('../Visualization/realRetweets.pickle', 'wb') as handle:
            pickle.dump(realRetweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
        with open('../Visualization/fakeTweets.pickle', 'wb') as handle:
            pickle.dump(fakeTweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
        with open('../Visualization/realTweets.pickle', 'wb') as handle:
            pickle.dump(realTweets, handle, protocol=pickle.HIGHEST_PROTOCOL)


        fakeArticlesPD.to_pickle("../Visualization/fakePD.pkl")
        realArticlesPD.to_pickle("../Visualization/realPD.pkl")
        authenticity = 'real'
    rootdir = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/'+authenticity
    
    for subdir, dirs, articles in os.walk(rootdir):
        for i,article in enumerate(dirs):
            #print(subdir)
            temp = pd.DataFrame()
            #path_to_json = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact11773/tweets' 
            path_to_json = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/' + authenticity + '/'+ article + '/tweets' 
            #print(path_to_json)
            json_pattern = os.path.join(path_to_json,'*.json')
            file_list = glob.glob(json_pattern)
            tweets_per_article.append(0)
            for file in file_list:
                #print(tweets_per_article)
                try:
                    data = pd.read_json(open(file, "r", encoding="utf8", errors="surrogateescape"), lines=True)
                    temp = temp.append(data, ignore_index = True)
                    tweets_per_article[i] += 1
                    #print(type(data['retweets'][0][0]))
                    #print(type(data))
                    if authenticity == 'fake':
                        fakeArticlesPD = fakeArticlesPD.append(data, ignore_index = True)
                    else:
                        realArticlesPD = realArticlesPD.append(data, ignore_index = True)
                except ValueError:
                    print("ValueError")
                except IndexError:
                    pass
                #print(fakeArticlesPD)
        #print(temp["user"][0]['id'])
        #print(temp['retweets'][13][0]['user']['id'])
            #print(temp.head())

        #FINN UNIKE VERDIER UNDER USER I TEMP: 
            users = list() #LAG LISTE HELLER
            texts = list()
            if "user" in temp.keys():
                for dic in temp["user"]:
                    if dic['id'] not in users:
                        users.append(dic['id'])
                        
            tweetUsers = list()
            texts = list()
            if "user" in temp.keys():
                for dic in temp["user"]:
                    if dic['id'] not in tweetUsers:
                        tweetUsers.append(dic['id'])
        
            temp = pd.DataFrame()        
        #path_to_json = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact11773/retweets' 
            path_to_json = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/'+authenticity+'/'+article+'/retweets' 
            #print(path_to_json)
            json_pattern = os.path.join(path_to_json,'*.json')
            file_list = glob.glob(json_pattern)

            for file in file_list:
            #print(file)
                try:
                    data = pd.read_json(open(file, "r", encoding="utf8", errors="surrogateescape"), lines=True)
                    temp = temp.append(data, ignore_index = True)
                    tweets_per_article[i] += 1
                    if authenticity == 'fake':
                        fakeArticlesPD = fakeArticlesPD.append(data['retweets'][0][0], ignore_index = True)
                    else:
                        realArticlesPD = realArticlesPD.append(data['retweets'][0][0], ignore_index = True)
                    break
                except ValueError:
                    print("ValueError")
                except IndexError:
                    pass
            if "retweets" in temp.keys():
                try:
                    print(temp['retweets'][0][0]['id'])
                except IndexError:
                    pass
        #print(temp["user"][0]['id'])
        #print(temp['retweets'][13][0]['user']['id'])
            #print(temp['retweets'][0]['id'])
        #FINN UNIKE VERDIER UNDER USER I TEMP: 
        #users = list() #LAG LISTE HELLER
            try:
                users.append(temp['retweets'][0][0]['id'])
            except IndexError:
                pass
            except KeyError:
                pass
            retweetUsers = list()                
            try:
                retweetUsers.append(temp['retweets'][0][0]['id'])
                print(retweetUsers)
            except IndexError:
                pass
            except KeyError:
                pass
            #Total (Retweets and Tweets)
            if i == 0:
            #Initialize dict
                for user in users:
                    if user not in fakeUsers.keys():
                        fakeUsers[user] = 0
            #increment usercounts
                for fakeUser in fakeUsers:
                    fakeUsers[fakeUser] = fakeUsers[fakeUser]+1
                #print(fakeUsers)
            elif i == 1:
                #Initialize dict
                for user in users:
                    if user not in realUsers.keys():
                        realUsers[user] = 0
                #increment usercounts
                for realUser in realUsers:
                    realUsers[realUser] = realUsers[realUser]+1
              
            #Retweets
            if i == 0:
            #Initialize dict
                for retweet in retweetUsers:
                    if retweet not in fakeRetweets.keys():
                        fakeRetweets[retweet] = 0
            #increment usercounts
                for fakeRetweet in fakeRetweets:
                    fakeRetweets[fakeRetweet] = fakeRetweets[fakeRetweet]+1
                #print(fakeUsers)
            elif i == 1:
                #Initialize dict
                for retweet in retweetUsers:
                    if retweet not in realRetweets.keys():
                        realRetweets[retweet] = 0
                #increment usercounts
                for realRetweet in realRetweets:
                    realRetweets[realRetweet] = realRetweets[realRetweet]+1
            
            #Tweets
            if i == 0:
            #Initialize dict
                for tweet in tweetUsers:
                    if tweet not in fakeTweets.keys():
                        fakeTweets[tweet] = 0
            #increment usercounts
                for fakeTweet in fakeTweets:
                    fakeTweets[fakeTweet] = fakeTweets[fakeTweet]+1
                #print(fakeUsers)
            elif i == 1:
                #Initialize dict
                for tweet in tweetUsers:
                    if tweet not in realTweets.keys():
                        realTweets[tweet] = 0
                #increment usercounts
                for realTweet in realTweets:
                    realTweets[realTweet] = realTweets[realTweet]+1

IndexError: list index out of range

In [21]:
import pickle
with open('tweets_per_article.pkl', 'wb') as f:
    pickle.dump(tweets_per_article, f)

with open('../Visualization/fakeUsers.pickle', 'wb') as handle:
    pickle.dump(fakeUsers, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Visualization/realUsers.pickle', 'wb') as handle:
    pickle.dump(realUsers, handle, protocol=pickle.HIGHEST_PROTOCOL)


with open('../Visualization/fakeRetweets.pickle', 'wb') as handle:
    pickle.dump(fakeRetweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Visualization/realRetweets.pickle', 'wb') as handle:
    pickle.dump(realRetweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
    
with open('../Visualization/fakeTweets.pickle', 'wb') as handle:
    pickle.dump(fakeTweets, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('../Visualization/realTweets.pickle', 'wb') as handle:
    pickle.dump(realTweets, handle, protocol=pickle.HIGHEST_PROTOCOL)


fakeArticlesPD.to_pickle("../Visualization/fakePD.pkl")
realArticlesPD.to_pickle("../Visualization/realPD.pkl")

## Process Data:

In [22]:
import pickle
fakeArticlesPD = pd.read_pickle('./fakeArticlesPD.pkl')
realArticlesPD = pd.read_pickle('./realArticlesPD.pkl')
#explode user column
exploded = realArticlesPD.user.apply(pd.Series)
exploded.columns = [str(col) + '_user' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='user'), exploded], axis=1)

exploded = realArticlesPD.entities.apply(pd.Series)
exploded.columns = [str(col) + '_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='entities'), exploded], axis=1)

exploded = realArticlesPD.quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='quoted_status'), exploded], axis=1)

exploded = realArticlesPD.entities_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_entities_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='entities_quoted_status'), exploded], axis=1)

exploded = realArticlesPD.entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_entities_user' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='entities_user'), exploded], axis=1)

exploded = realArticlesPD.description_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_description_entities_user' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='description_entities_user'), exploded], axis=1)

exploded = realArticlesPD.hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_hashtags_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='hashtags_entities'), exploded], axis=1)

#change name of 0 to zero
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('0_hashtags_entities')] = 'zero_hashtags_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.zero_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_zero_hashtags_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='zero_hashtags_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('1_hashtags_entities')] = 'one_hashtags_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.one_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_one_hashtags_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='one_hashtags_entities'), exploded], axis=1)

#change name of 2 to two
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('2_hashtags_entities')] = 'two_hashtags_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.two_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_two_hashtags_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='two_hashtags_entities'), exploded], axis=1)

#change name of 3 to three
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('3_hashtags_entities')] = 'three_hashtags_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.three_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_three_hashtags_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='three_hashtags_entities'), exploded], axis=1)

#change name of 4 to four
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('4_hashtags_entities')] = 'four_hashtags_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.four_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_four_hashtags_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='four_hashtags_entities'), exploded], axis=1)

#change name of 5 to five
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('5_hashtags_entities')] = 'five_hashtags_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.five_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_five_hashtags_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='five_hashtags_entities'), exploded], axis=1)

#explode urls_entities_quoted_status
exploded = realArticlesPD.urls_entities_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_urls_entities_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='urls_entities_quoted_status'), exploded], axis=1)

#change name of 0 to zero
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('0_urls_entities_quoted_status')] = 'zero_urls_entities_quoted_status';
realArticlesPD.columns = temp

exploded = realArticlesPD.zero_urls_entities_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls_entities_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='zero_urls_entities_quoted_status'), exploded], axis=1)

exploded = realArticlesPD.url_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_url_entities_user' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='url_entities_user'), exploded], axis=1)

exploded = realArticlesPD.urls_url_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_urls_url_entities_user' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='urls_url_entities_user'), exploded], axis=1)

#change name of 0 to zero
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('0_urls_url_entities_user')] = 'zero_urls_url_entities_user';
realArticlesPD.columns = temp

exploded = realArticlesPD.zero_urls_url_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls_url_entities_user' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='zero_urls_url_entities_user'), exploded], axis=1)

exploded = realArticlesPD.urls_entities.apply(pd.Series)
exploded.columns = [str(col) + '_urls_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='urls_entities'), exploded], axis=1)

#change name of 0 to zero
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('0_urls_entities')] = 'zero_urls_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.zero_urls_entities.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='zero_urls_entities'), exploded], axis=1)




#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('1_urls_entities')] = 'one_urls_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.one_urls_entities.apply(pd.Series)
exploded.columns = [str(col) + '_one_urls_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='one_urls_entities'), exploded], axis=1)

exploded = realArticlesPD.user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('0_user_mentions_entities')] = 'zero_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.zero_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_zero_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='zero_user_mentions_entities'), exploded], axis=1)


#explode user column
exploded = fakeArticlesPD.user.apply(pd.Series)
exploded.columns = [str(col) + '_user' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='user'), exploded], axis=1)

exploded = fakeArticlesPD.entities.apply(pd.Series)
exploded.columns = [str(col) + '_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='entities'), exploded], axis=1)

exploded = fakeArticlesPD.quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_quoted_status' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='quoted_status'), exploded], axis=1)

exploded = fakeArticlesPD.entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_entities_user' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='entities_user'), exploded], axis=1)

exploded = fakeArticlesPD.description_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_description_entities_user' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='description_entities_user'), exploded], axis=1)

exploded = fakeArticlesPD.hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='hashtags_entities'), exploded], axis=1)

#change name of 0 to zero
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('0_hashtags_entities')] = 'zero_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.zero_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_zero_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='zero_hashtags_entities'), exploded], axis=1)

#change name of 1 to zero
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('1_hashtags_entities')] = 'one_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.one_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_one_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='one_hashtags_entities'), exploded], axis=1)

#change name of 2 to two
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('2_hashtags_entities')] = 'two_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.two_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_two_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='two_hashtags_entities'), exploded], axis=1)

#change name of 3 to three
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('3_hashtags_entities')] = 'three_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.three_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_three_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='three_hashtags_entities'), exploded], axis=1)

#change name of 4 to four
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('4_hashtags_entities')] = 'four_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.four_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_four_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='four_hashtags_entities'), exploded], axis=1)

#change name of 5 to five
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('5_hashtags_entities')] = 'five_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.five_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_five_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='five_hashtags_entities'), exploded], axis=1)

#change name of 6 to six
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('6_hashtags_entities')] = 'six_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.six_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_six_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='six_hashtags_entities'), exploded], axis=1)

#change name of 7 to seven
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('7_hashtags_entities')] = 'seven_hashtags_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.seven_hashtags_entities.apply(pd.Series)
exploded.columns = [str(col) + '_seven_hashtags_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='seven_hashtags_entities'), exploded], axis=1)

exploded = fakeArticlesPD.url_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_url_entities_user' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='url_entities_user'), exploded], axis=1)

exploded = fakeArticlesPD.urls_url_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_urls_url_entities_user' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='urls_url_entities_user'), exploded], axis=1)

#change name of 0 to zero
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('0_urls_url_entities_user')] = 'zero_urls_url_entities_user';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.zero_urls_url_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls_url_entities_user' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='zero_urls_url_entities_user'), exploded], axis=1)

exploded = fakeArticlesPD.urls_entities.apply(pd.Series)
exploded.columns = [str(col) + '_urls_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='urls_entities'), exploded], axis=1)

#change name of 0 to zero
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('0_urls_entities')] = 'zero_urls_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.zero_urls_entities.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='zero_urls_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('1_urls_entities')] = 'one_urls_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.one_urls_entities.apply(pd.Series)
exploded.columns = [str(col) + '_one_urls_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='one_urls_entities'), exploded], axis=1)

exploded = fakeArticlesPD.user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('0_user_mentions_entities')] = 'zero_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.zero_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_zero_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='zero_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('1_user_mentions_entities')] = 'one_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.one_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_one_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='one_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('1_user_mentions_entities')] = 'one_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.one_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_one_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='one_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('2_user_mentions_entities')] = 'two_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.two_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_two_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='two_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('2_user_mentions_entities')] = 'two_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.two_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_two_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='two_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('3_user_mentions_entities')] = 'three_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.three_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_three_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='three_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('3_user_mentions_entities')] = 'three_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.three_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_three_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='three_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('4_user_mentions_entities')] = 'four_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.four_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_four_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='four_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('4_user_mentions_entities')] = 'four_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.four_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_four_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='four_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('5_user_mentions_entities')] = 'five_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.five_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_five_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='five_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('5_user_mentions_entities')] = 'five_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.five_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_five_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='five_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('6_user_mentions_entities')] = 'six_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.six_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_six_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='six_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('6_user_mentions_entities')] = 'six_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.six_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_six_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='six_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('7_user_mentions_entities')] = 'seven_user_mentions_entities';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.seven_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_seven_user_mentions_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='seven_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('7_user_mentions_entities')] = 'seven_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.seven_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_seven_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='seven_user_mentions_entities'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('8_user_mentions_entities')] = 'eight_user_mentions_entities';
realArticlesPD.columns = temp

exploded = realArticlesPD.eight_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_eight_user_mentions_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='eight_user_mentions_entities'), exploded], axis=1)

exploded = realArticlesPD.user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_user_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='user_quoted_status'), exploded], axis=1)

exploded = fakeArticlesPD.user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_user_quoted_status' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='user_quoted_status'), exploded], axis=1)

exploded = realArticlesPD.extended_entities.apply(pd.Series)
exploded.columns = [str(col) + '_extended_entities' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='extended_entities'), exploded], axis=1)

exploded = fakeArticlesPD.extended_entities.apply(pd.Series)
exploded.columns = [str(col) + '_extended_entities' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='extended_entities'), exploded], axis=1)

exploded = realArticlesPD.entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_entities_user_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='entities_user_quoted_status'), exploded], axis=1)

exploded = fakeArticlesPD.entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_entities_user_quoted_status' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='entities_user_quoted_status'), exploded], axis=1)

exploded = realArticlesPD.description_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_description_entities_user_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='description_entities_user_quoted_status'), exploded], axis=1)

exploded = fakeArticlesPD.description_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_description_entities_user_quoted_status' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='description_entities_user_quoted_status'), exploded], axis=1)

exploded = realArticlesPD.url_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_url_entities_user_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='url_entities_user_quoted_status'), exploded], axis=1)

exploded = fakeArticlesPD.url_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_url_entities_user_quoted_status' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='url_entities_user_quoted_status'), exploded], axis=1)

exploded = realArticlesPD.urls_url_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_urls__url_entities_user_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='urls_url_entities_user_quoted_status'), exploded], axis=1)

exploded = fakeArticlesPD.urls_url_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_urls_url_entities_user_quoted_status' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='urls_url_entities_user_quoted_status'), exploded], axis=1)

#change name of 1 to one
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('0_urls__url_entities_user_quoted_status')] = 'zero_urls__url_entities_user_quoted_status';
realArticlesPD.columns = temp

exploded = realArticlesPD.zero_urls__url_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls__url_entities_user_quoted_status' for col in exploded.columns]
realArticlesPD = pd.concat([realArticlesPD.drop(columns='zero_urls__url_entities_user_quoted_status'), exploded], axis=1)

#change name of 1 to one
temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('0_urls_url_entities_user_quoted_status')] = 'zero_urls_url_entities_user_quoted_status';
fakeArticlesPD.columns = temp

exploded = fakeArticlesPD.zero_urls_url_entities_user_quoted_status.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls_url_entities_user_quoted_status' for col in exploded.columns]
fakeArticlesPD = pd.concat([fakeArticlesPD.drop(columns='zero_urls_url_entities_user_quoted_status'), exploded], axis=1)


#0_urls_url_entities_user_quoted_status

#add label to data
realArticlesPD['label'] = 'real'
#add label to data
fakeArticlesPD['label'] = 'fake'

## Read Articles

In [ ]:
with open('tweets_per_article.pkl', 'rb') as f:
    tweets_per_article = pickle.load(f)

#change name of text to user_text
temp = realArticlesPD.columns.values; 
temp[realArticlesPD.columns.get_loc('text')] = 'hashtags_text';
realArticlesPD.columns = temp

temp = fakeArticlesPD.columns.values; 
temp[fakeArticlesPD.columns.get_loc('text')] = 'hashtags_text';
fakeArticlesPD.columns = temp

In [68]:
fakePD = pd.DataFrame()
realPD = pd.DataFrame()

for i in range(2):
    if i == 0:
        authenticity = 'fake'
    else:
        authenticity = 'real'
    rootdir = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/'+authenticity
    
    for subdir, dirs, articles in os.walk(rootdir):
        for article in dirs:
            temp = pd.DataFrame()
            path_to_json = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/' + authenticity + '/'+ article 
            json_pattern = os.path.join(path_to_json,'*.json')
            file_list = glob.glob(json_pattern)
            for file in file_list:
                #print(file)
                try:
                    data = pd.read_json(open(file, "r", encoding="utf8", errors="surrogateescape"), lines=True)
                    if authenticity == 'fake':
                        fakePD = fakePD.append(data, ignore_index = True)
                    else:
                        realPD = realPD.append(data, ignore_index = True)
                except ValueError:
                    print("ValueError")
                    
                    
            for art in file_list:
                path_to_json = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/' + authenticity + '/'+ article 
                json_pattern = os.path.join(path_to_json,'*.json')
                file_list = glob.glob(json_pattern)
                for i, file in enumerate(file_list):
                    try:
                        data = pd.read_json(open(file, "r", encoding="utf8", errors="surrogateescape"), lines=True)
                        if authenticity == 'fake':
                            for tweet in range(tweets_per_article[i]):
                                fakeArticlesPD = fakeArticlesPD.append(data, ignore_index = True)
                        else:
                            for tweet in range(tweets_per_article[i]):
                                realArticlesPD = realArticlesPD.append(data, ignore_index = True)
                    except ValueError:
                        print("ValueError")

C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact11773\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13038\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13467\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13468\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13475\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13484\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13496\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13501\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13515\news conte

C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13987\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact13999\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14003\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14005\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14008\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14021\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14026\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14040\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14043\news conte

C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14395\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14404\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14406\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14408\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14417\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14421\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14426\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14436\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14447\news conte

C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14777\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14785\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14788\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14789\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14794\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14795\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14810\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14811\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact14814\news conte

C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15158\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15159\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15161\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15164\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15178\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15187\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15188\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15190\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15191\news conte

C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15501\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15505\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15512\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15514\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15525\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15532\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15533\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15534\news content.json
C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/politifact/fake/politifact15539\news conte

KeyboardInterrupt: 

In [37]:
realArticlesPD.shape

(1383, 264)

In [10]:
import pickle
fakePD.to_pickle("../Visualization/fakePD.pkl")
realPD.to_pickle("../Visualization/realPD.pkl")

## Append Tweets/Retweets and Articles

In [5]:
#combine PDs
bigdata = realArticlesPD.append(fakeArticlesPD, ignore_index=True)

#drop zero non-null columns:
bigdata = bigdata.dropna(axis = 1, how = 'all')

#drop rows with less than 63 non-NAN values:
bigdata = bigdata.dropna(thresh=7) #t = 0 gives 1594 entries, t = 5 gives 1592 entries, t=10 gives 793 entries

#movel label to end:
#cols = list(df.columns.values) #Make a list of all of the columns in the df
#cols.pop(cols.index('b')) #Remove b from list
#cols.pop(cols.index('x')) #Remove x from list
#df = df[cols+['b','x']]

#display head
bigdata.head()

bigdata.to_pickle("dummy.pkl")

## Read User Followers and User Following

In [3]:
import os, json
from os import path
import pandas as pd
import numpy as np
import glob
user_followers = pd.DataFrame()
user_following = pd.DataFrame()
for i in range(2):
    if i == 0:
        folder = 'user_followers'
    else:
        folder = 'user_following'
    rootdir = 'C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/'+folder
    json_pattern = os.path.join(rootdir,'*.json')
    file_list = glob.glob(json_pattern)

    for file in file_list:
        #print(file)
        data = pd.read_json(open(file, "r", encoding="utf8", errors="surrogateescape"), lines=True)
        if i==0:
            user_followers = user_followers.append(data, ignore_index = True)
        else:
            user_following = user_following.append(data, ignore_index = True)
            
user_followers.to_pickle("dummy_followers.pkl")
user_following.to_pickle("dummy_following.pkl")

MemoryError: 

## Append Followers/Following

In [ ]:
#import pandas as pd
#user_followers = pd.read_pickle('./dummy_followers.pkl')
#user_following = pd.read_pickle('./dummy_following.pkl')

## Explode new columns, rename followers_x and followers_y, and move label to end

In [2]:
import pandas as pd
bigdata = pd.read_pickle('dummy.pkl')

exploded = bigdata.urls_description_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_urls_description_entities_user' for col in exploded.columns]
bigdata = pd.concat([bigdata.drop(columns='urls_description_entities_user'), exploded], axis=1)

#change name of 1 to one
temp = bigdata.columns.values; 
temp[bigdata.columns.get_loc('0_urls_description_entities_user')] = 'zero_urls_description_entities_user';
bigdata.columns = temp

exploded = bigdata.zero_urls_description_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_zero_urls_description_entities_user' for col in exploded.columns]
bigdata = pd.concat([bigdata.drop(columns='zero_urls_description_entities_user'), exploded], axis=1)

#flatten indices and symbols_entities
exploded = bigdata.indices_zero_urls_entities.apply(pd.Series)
exploded.columns = [str(col) + '_indices_zero_urls_entities' for col in exploded.columns]
bigdata = pd.concat([bigdata.drop(columns='indices_zero_urls_entities'), exploded], axis=1)
temp = bigdata.columns.values; 
temp[bigdata.columns.get_loc('0_indices_zero_urls_entities')] = 'zero_indices_zero_urls_entities';
temp[bigdata.columns.get_loc('1_indices_zero_urls_entities')] = 'one_indices_zero_urls_entities';
bigdata.columns = temp

exploded = bigdata.indices_zero_urls_url_entities_user.apply(pd.Series)
exploded.columns = [str(col) + '_indices_zero_urls_url_entities_user' for col in exploded.columns]
bigdata = pd.concat([bigdata.drop(columns='indices_zero_urls_url_entities_user'), exploded], axis=1)
temp = bigdata.columns.values; 
temp[bigdata.columns.get_loc('0_indices_zero_urls_url_entities_user')] = 'zero_indices_zero_urls_url_entities_user';
temp[bigdata.columns.get_loc('1_indices_zero_urls_url_entities_user')] = 'one_indices_zero_urls_url_entities_user';
bigdata.columns = temp

exploded = bigdata.indices_zero_user_mentions_entities.apply(pd.Series)
exploded.columns = [str(col) + '_indices_zero_user_mentions_entities' for col in exploded.columns]
bigdata = pd.concat([bigdata.drop(columns='indices_zero_user_mentions_entities'), exploded], axis=1)
temp = bigdata.columns.values; 
temp[bigdata.columns.get_loc('0_indices_zero_user_mentions_entities')] = 'zero_indices_zero_user_mentions_entities';
temp[bigdata.columns.get_loc('1_indices_zero_user_mentions_entities')] = 'one_indices_zero_user_mentions_entities';
bigdata.columns = temp

exploded = bigdata.symbols_entities.apply(pd.Series)
exploded.columns = [str(col) + '_symbols_entities' for col in exploded.columns]
bigdata = pd.concat([bigdata.drop(columns='symbols_entities'), exploded], axis=1)

user_followers = pd.read_pickle('./dummy_followers.pkl')
user_following = pd.read_pickle('./dummy_following.pkl')
user_followers.user_id = user_followers.user_id.astype(str)
user_following.user_id = user_following.user_id.astype(str)
user_followers = user_followers.rename(columns={'user_id': 'id_str_user'})
user_following = user_following.rename(columns={'user_id': 'id_str_user'})
bigdata = pd.merge(bigdata, user_followers, on='id_str_user')
bigdata = pd.merge(bigdata, user_following, on='id_str_user')
bigdata.head()

bigdata = bigdata.rename(columns={'followers_x': 'followers'})
bigdata = bigdata.rename(columns={'followers_y': 'following'})

#move label to last column
df1 = bigdata.pop('label') # remove column b and store it in df1
bigdata['label']=df1 # add b series as a 'new' column.

bigdata.to_pickle("bigdata.pkl")

In [79]:
import pandas as pd
bigdata = pd.read_pickle("bigdata.pkl")

#drop zero non-null columns:
bigdata = bigdata.dropna(axis = 1, how = 'all')

#drop rows with less than 63 non-NAN values:
bigdata = bigdata.dropna(thresh=7) #t = 0 gives 1594 entries, t = 5 gives 1592 entries, t=10 gives 793 entries
bigdata.to_pickle("bigdata.pkl")

In [80]:
import pandas as pd
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
bigdataPKL = pd.read_pickle("bigdata.pkl")
bigdataPKL.tail()

,6_hashtags_entities,9_user_mentions_entities,contributors_enabled_user,contributors_enabled_user_quoted_status,coordinates,created_at,created_at_quoted_status,created_at_user,created_at_user_quoted_status,default_profile_image_user,default_profile_image_user_quoted_status,default_profile_user,default_profile_user_quoted_status,description_user,description_user_quoted_status,display_url_one_urls_entities,display_url_zero_urls__url_entities_user_quoted_status,display_url_zero_urls_entities,display_url_zero_urls_entities_quoted_status,display_url_zero_urls_url_entities_user,display_url_zero_urls_url_entities_user_quoted_status,entities_quoted_status,expanded_url_one_urls_entities,expanded_url_zero_urls__url_entities_user_quoted_status,expanded_url_zero_urls_entities,expanded_url_zero_urls_entities_quoted_status,expanded_url_zero_urls_url_entities_user,expanded_url_zero_urls_url_entities_user_quoted_status,extended_entities_quoted_status,favorite_count,favorite_count_quoted_status,favorited,favorited_quoted_status,favourites_count_user,favourites_count_user_quoted_status,follow_request_sent_user,follow_request_sent_user_quoted_status,followers_count_user,followers_count_user_quoted_status,following_user,following_user_quoted_status,friends_count_user,friends_count_user_quoted_status,geo,geo_enabled_user,geo_enabled_user_quoted_status,has_extended_profile_user,has_extended_profile_user_quoted_status,hashtags_entities_quoted_status,id,id_eight_user_mentions_entities,id_five_user_mentions_entities,id_four_user_mentions_entities,id_one_user_mentions_entities,id_quoted_status,id_seven_user_mentions_entities,id_six_user_mentions_entities,id_str,id_str_eight_user_mentions_entities,id_str_five_user_mentions_entities,id_str_four_user_mentions_entities,id_str_one_user_mentions_entities,id_str_quoted_status,id_str_seven_user_mentions_entities,id_str_six_user_mentions_entities,id_str_three_user_mentions_entities,id_str_two_user_mentions_entities,id_str_user,id_str_user_quoted_status,id_str_zero_user_mentions_entities,id_three_user_mentions_entities,id_two_user_mentions_entities,id_user,id_user_quoted_status,id_zero_user_mentions_entities,in_reply_to_screen_name,in_reply_to_screen_name_quoted_status,in_reply_to_status_id,in_reply_to_status_id_quoted_status,in_reply_to_status_id_str,in_reply_to_status_id_str_quoted_status,in_reply_to_user_id,in_reply_to_user_id_quoted_status,in_reply_to_user_id_str,in_reply_to_user_id_str_quoted_status,indices_eight_user_mentions_entities,indices_five_hashtags_entities,indices_five_user_mentions_entities,indices_four_hashtags_entities,indices_four_user_mentions_entities,indices_one_hashtags_entities,indices_one_urls_entities,indices_one_user_mentions_entities,indices_seven_hashtags_entities,indices_seven_user_mentions_entities,indices_six_hashtags_entities,indices_six_user_mentions_entities,indices_three_hashtags_entities,indices_three_user_mentions_entities,indices_two_hashtags_entities,indices_two_user_mentions_entities,indices_zero_hashtags_entities,indices_zero_urls__url_entities_user_quoted_status,indices_zero_urls_entities_quoted_status,indices_zero_urls_url_entities_user_quoted_status,is_quote_status,is_quote_status_quoted_status,is_translation_enabled_user,is_translation_enabled_user_quoted_status,is_translator_user,is_translator_user_quoted_status,lang,lang_quoted_status,listed_count_user,listed_count_user_quoted_status,location_user,location_user_quoted_status,media_entities,media_entities_quoted_status,media_extended_entities,name_eight_user_mentions_entities,name_five_user_mentions_entities,name_four_user_mentions_entities,name_one_user_mentions_entities,name_seven_user_mentions_entities,name_six_user_mentions_entities,name_three_user_mentions_entities,name_two_user_mentions_entities,name_user,name_user_quoted_status,name_zero_user_mentions_entities,notifications_user,notifications_user_quoted_status,place,possibly_sensitive,possibly_sensitive_appealable,possibly_sensitive_appealable_quoted_status,

In [81]:
#remove nan columns by threshold
bigdataPKL = bigdataPKL.dropna(axis = 1, thresh=98)
bigdataPKL.head()

,contributors_enabled_user,created_at,created_at_user,default_profile_image_user,default_profile_user,description_user,display_url_zero_urls_entities,display_url_zero_urls_url_entities_user,expanded_url_zero_urls_entities,expanded_url_zero_urls_url_entities_user,favorite_count,favorited,favourites_count_user,follow_request_sent_user,followers_count_user,following_user,friends_count_user,geo_enabled_user,has_extended_profile_user,id,id_str,id_str_user,id_str_zero_user_mentions_entities,id_user,id_zero_user_mentions_entities,is_quote_status,is_translation_enabled_user,is_translator_user,lang,listed_count_user,location_user,name_user,name_zero_user_mentions_entities,notifications_user,possibly_sensitive,possibly_sensitive_appealable,profile_background_color_user,profile_background_image_url_https_user,profile_background_image_url_user,profile_background_tile_user,profile_banner_url_user,profile_image_url_https_user,profile_image_url_user,profile_link_color_user,profile_sidebar_border_color_user,profile_sidebar_fill_color_user,profile_text_color_user,profile_use_background_image_user,protected_user,retweet_count,retweeted,screen_name_user,screen_name_zero_user_mentions_entities,source,statuses_count_user,text,translator_type_user,truncated,url_user,url_zero_urls_entities,url_zero_urls_url_entities_user,verified_user,followers,following,zero_indices_zero_urls_entities,one_indices_zero_urls_entities,zero_indices_zero_urls_url_entities_user,one_indices_zero_urls_url_entities_user,zero_indices_zero_user_mentions_entities,one_indices_zero_user_mentions_entities,label
0,False,2015-10-09 21:18:53+00:00,Fri Jun 27 23:15:54 +0000 2014,False,False,Sagittarius♐️ | @darrencriss🕶 | Demi Lovato💖 |...,eventbrite.com/e/derry-town-h…,NaN,https://www.eventbrite.com/e/derry-town-hall-m...,NaN,1.0,False,3936.0,False,632.0,False,950.0,True,True,6.525941e+17,6.525941e+17,2592090978,15745368,2.592091e+09,15745368.0,False,False,False,en,14.0,"Jericho, NY",Paige Johnson,Marco Rubio,False,False,False,2F0EEB,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_banners/25920909...,https://pbs.twimg.com/profile_images/112189326...,http://pbs.twimg.com/profile_images/1121893261...,ED0E0E,FFFFFF,DDEEF6,333333,True,False,2.0,False,Paige_Johnson17,marcorubio,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",3109.0,Missed @marcorubio in NH this week? Come to a ...,none,False,None,https://t.co/FlrKPldQ76,NaN,False,[],[],117.0,140.0,NaN,NaN,7.0,18.0,real
1,False,2013-03-22 06:36:15+00:00,Tue Nov 10 15:44:34 +0000 2009,False,False,Latest bill activity in the Michigan Legislature,1.usa.gov/15nzBw1,legislature.mi.gov,http://1.usa.gov/15nzBw1,http://www.legislature.mi.gov,0.0,False,1.0,False,1370.0,False,0.0,False,False,3.149889e+17,3.149889e+17,88953774,NaN,8.895377e+07,NaN,False,False,False,en,68.0,"Lansing, Michigan",Michigan Legislature,NaN,False,False,False,FFFFFF,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,False,NaN,https://pbs.twimg.com/profile_images/519651617...,http://pbs.twimg.com/profile_images/519651617/...,0084B4,C0DEED,E5D7C8,333333,True,False,0.0,False,michiganleg,NaN,"<a href=""http://www.legislature.mi.gov"" rel=""n...",84579.0,2013-HB-4469 Public employees and officers; et...,none,False,http://t.co/7t82uxHYgJ,http://t.co/8qY8mG0LMT,http://t.co/7t82uxHYgJ,False,[],[],91.0,113.0,0.0,22.0,NaN,NaN,real
2,False,2008-03-03 06:10:37+00:00,Wed Mar 21 16:48:44 +0000 2007,False,False,Breaking news and features from the National D...,NaN,nytimes.com/national,NaN,http://nytimes.com/national,0.0,False,2010.0,False,79814.0,False,820.0,True,False,7.659183e+08,7.659183e+08,1767741,NaN,1.767741e+06,NaN,False,False,False,en,2882.0,From Sea to Shining Sea,NYT National News,NaN,False,NaN,NaN,FFFFFF,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_b

In [83]:
#badne7 - gorina6 - jupyter-lab start, gorina6.ux.uis.no:8890/lab i web-browser, token - jupyter notebook list
#C:/Users/henri/FakenewsData/fakenewsnet_data/fakenewsnet_data/
#Convert IDs to Strings:
import collections
import numpy as np
#bigdataPKL['indices_zero_urls_entities'] = bigdataPKL['indices_zero_urls_entities'].astype(str)
#bigdataPKL['indices_zero_urls_url_entities_user'] = bigdataPKL['indices_zero_urls_url_entities_user'].astype(str)
#bigdataPKL['indices_zero_user_mentions_entities'] = bigdataPKL['indices_zero_user_mentions_entities'].astype(str)
bigdataPKL['followers'] = bigdataPKL['followers'].astype(str)
bigdataPKL['following'] = bigdataPKL['following'].astype(str)
#bigdataPKL['symbols_entities'] = bigdataPKL['symbols_entities'].astype(str)
for col in bigdataPKL.columns:
    if(bigdataPKL[col].dtype == bool):
        bigdataPKL[col].fillna('Unknown', inplace=True)
    elif(bigdataPKL[col].dtype == np.float64):
        bigdataPKL[col].fillna(bigdataPKL[col].mean(), inplace=True)
    elif(bigdataPKL[col].dtype == object):
        #Replace with most common value for rows class
        fakeIndexes = bigdataPKL.index[bigdataPKL['label'] == 'fake'].tolist()
        realIndexes = bigdataPKL.index[bigdataPKL['label'] == 'real'].tolist()
        realVals = []
        fakeVals = []
        for idx in fakeIndexes:
            fakeVals.append(bigdataPKL[col][idx])
        for idx in realIndexes:
            realVals.append(bigdataPKL[col][idx])
        #print(collections.Counter(fakeVals)[0])
        for fakeID in fakeIndexes:
            try:
                if np.isnan(np.float(bigdataPKL[col][fakeID])):
                    while 'nan' in fakeVals: 
                        fakeVals.remove('nan')  
                    while 'NaN' in fakeVals: 
                        fakeVals.remove('NaN') 
                    #print(fakeID)
                    bigdataPKL[col][fakeID] = collections.Counter(fakeVals).most_common()[0][0]
            except ValueError:
                pass
            except TypeError:
                pass
        for realID in realIndexes:
            try:
                if np.isnan(np.float(bigdataPKL[col][realID])):
                    while 'nan' in realVals: 
                        realVals.remove('nan') 
                    while 'NaN' in realVals: 
                        realVals.remove('NaN')  
                    #print(realID)
                    #print(collections.Counter(realVals).most_common()[0][0])
                    bigdataPKL[col][realID] = collections.Counter(realVals).most_common()[0][0]
            except ValueError:
                pass
            except TypeError:
                pass
bigdataPKL.head()
#se twitter developer docs
#prøv forskjellige thresh og drop rows/columns
#string/url - convert nan to most common value of fake/real records
#strip away indice brackets using clean method
##fjern unødvendige kolonner
#last opp til github

#https://towardsdatascience.com/fake-news-classification-using-glove-and-long-short-term-memory-lstm-a48f1dd605ab

#user-user_entities-quoted_status-quoted_status_user-quoted_status_user_entities, entities-hashtags..
#print(bigdata.columns[bigdata.columns.duplicated()])

C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,contributors_enabled_user,created_at,created_at_user,default_profile_image_user,default_profile_user,description_user,display_url_zero_urls_entities,display_url_zero_urls_url_entities_user,expanded_url_zero_urls_entities,expanded_url_zero_urls_url_entities_user,favorite_count,favorited,favourites_count_user,follow_request_sent_user,followers_count_user,following_user,friends_count_user,geo_enabled_user,has_extended_profile_user,id,id_str,id_str_user,id_str_zero_user_mentions_entities,id_user,id_zero_user_mentions_entities,is_quote_status,is_translation_enabled_user,is_translator_user,lang,listed_count_user,location_user,name_user,name_zero_user_mentions_entities,notifications_user,possibly_sensitive,possibly_sensitive_appealable,profile_background_color_user,profile_background_image_url_https_user,profile_background_image_url_user,profile_background_tile_user,profile_banner_url_user,profile_image_url_https_user,profile_image_url_user,profile_link_color_user,profile_sidebar_border_color_user,profile_sidebar_fill_color_user,profile_text_color_user,profile_use_background_image_user,protected_user,retweet_count,retweeted,screen_name_user,screen_name_zero_user_mentions_entities,source,statuses_count_user,text,translator_type_user,truncated,url_user,url_zero_urls_entities,url_zero_urls_url_entities_user,verified_user,followers,following,zero_indices_zero_urls_entities,one_indices_zero_urls_entities,zero_indices_zero_urls_url_entities_user,one_indices_zero_urls_url_entities_user,zero_indices_zero_user_mentions_entities,one_indices_zero_user_mentions_entities,label
0,False,2015-10-09 21:18:53+00:00,Fri Jun 27 23:15:54 +0000 2014,False,False,Sagittarius♐️ | @darrencriss🕶 | Demi Lovato💖 |...,eventbrite.com/e/derry-town-h…,usdemocrats.com,https://www.eventbrite.com/e/derry-town-hall-m...,None,1.0,False,3936.0,False,632.0,False,950.0,True,True,6.525941e+17,6.525941e+17,2592090978,15745368,2.592091e+09,1.574537e+07,False,False,False,en,14.0,"Jericho, NY",Paige Johnson,Marco Rubio,False,False,False,2F0EEB,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_banners/25920909...,https://pbs.twimg.com/profile_images/112189326...,http://pbs.twimg.com/profile_images/1121893261...,ED0E0E,FFFFFF,DDEEF6,333333,True,False,2.0,False,Paige_Johnson17,marcorubio,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",3109.0,Missed @marcorubio in NH this week? Come to a ...,none,False,None,https://t.co/FlrKPldQ76,http://t.co/DFfnwVB1mD,False,[],[],117.000000,140.000000,0.0,22.545205,7.000000,18.000000,real
1,False,2013-03-22 06:36:15+00:00,Tue Nov 10 15:44:34 +0000 2009,False,False,Latest bill activity in the Michigan Legislature,1.usa.gov/15nzBw1,legislature.mi.gov,http://1.usa.gov/15nzBw1,http://www.legislature.mi.gov,0.0,False,1.0,False,1370.0,False,0.0,False,False,3.149889e+17,3.149889e+17,88953774,8953122,8.895377e+07,8.529181e+16,False,False,False,en,68.0,"Lansing, Michigan",Michigan Legislature,PolitiFact,False,False,False,FFFFFF,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/18188773...,https://pbs.twimg.com/profile_images/519651617...,http://pbs.twimg.com/profile_images/519651617/...,0084B4,C0DEED,E5D7C8,333333,True,False,0.0,False,michiganleg,PolitiFact,"<a href=""http://www.legislature.mi.gov"" rel=""n...",84579.0,2013-HB-4469 Public employees and officers; et...,none,False,http://t.co/7t82uxHYgJ,http://t.co/8qY8mG0LMT,http://t.co/7t82uxHYgJ,False,[],[],91.000000,113.000000,0.0,22.000000,49.264865,60.043243,real
2,False,2008-03-03 06:10:37+00:00,Wed Mar 21 16:48:44 +0000 2007,False,False,Breaking news and features from the National D...,twitter.com/i/web/status/1…,nytimes.com/national,http://aol.it/ZqFuvl,http://nytimes.com/national,0.0,False,2010.0,False,79814.0,False,820.0,True,False,7.659183e+08,7.659183e+08,1767741,8953122,1.767741e+06,8.529181e+16,False,False,Fals

In [84]:
try:
    for row in bigdataPKL['following'].keys(): 
        bigdataPKL['following'][row] = bigdataPKL['following'][row].strip('][')
    for row in bigdataPKL['followers'].keys():
        bigdataPKL['followers'][row] = bigdataPKL['followers'][row].strip('][')
except ValueError:
    pass
bigdataPKL.tail()

C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,contributors_enabled_user,created_at,created_at_user,default_profile_image_user,default_profile_user,description_user,display_url_zero_urls_entities,display_url_zero_urls_url_entities_user,expanded_url_zero_urls_entities,expanded_url_zero_urls_url_entities_user,favorite_count,favorited,favourites_count_user,follow_request_sent_user,followers_count_user,following_user,friends_count_user,geo_enabled_user,has_extended_profile_user,id,id_str,id_str_user,id_str_zero_user_mentions_entities,id_user,id_zero_user_mentions_entities,is_quote_status,is_translation_enabled_user,is_translator_user,lang,listed_count_user,location_user,name_user,name_zero_user_mentions_entities,notifications_user,possibly_sensitive,possibly_sensitive_appealable,profile_background_color_user,profile_background_image_url_https_user,profile_background_image_url_user,profile_background_tile_user,profile_banner_url_user,profile_image_url_https_user,profile_image_url_user,profile_link_color_user,profile_sidebar_border_color_user,profile_sidebar_fill_color_user,profile_text_color_user,profile_use_background_image_user,protected_user,retweet_count,retweeted,screen_name_user,screen_name_zero_user_mentions_entities,source,statuses_count_user,text,translator_type_user,truncated,url_user,url_zero_urls_entities,url_zero_urls_url_entities_user,verified_user,followers,following,zero_indices_zero_urls_entities,one_indices_zero_urls_entities,zero_indices_zero_urls_url_entities_user,one_indices_zero_urls_url_entities_user,zero_indices_zero_user_mentions_entities,one_indices_zero_user_mentions_entities,label
788,False,2017-05-20 14:11:39+00:00,Sun Oct 25 14:01:48 +0000 2009,False,True,,fb.me/1fYsjNaLC,thekind-heartedsmartaleck.blogspot.com,http://fb.me/1fYsjNaLC,http://thekind-heartedsmartaleck.blogspot.com/,0.0,False,16.0,False,47.0,False,87.0,True,True,8.659330e+17,8.659330e+17,85085807,10228272,8.508581e+07,8.529181e+16,False,False,False,en,7.0,Gassaway WV,Terry Bly,YouTube,False,False,False,C0DEED,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/13552184...,https://pbs.twimg.com/profile_images/249393875...,http://pbs.twimg.com/profile_images/2493938757...,1DA1F2,C0DEED,DDEEF6,333333,True,False,0.0,False,TlbBly,YouTube,"<a href=""http://www.facebook.com/twitter"" rel=...",90486.0,WORSE THAN HITLER! Trey Gowdy’s Son Found In A...,none,False,None,https://t.co/dxpndRqHjL,http://t.co/PYrnTHpCvI,False,"1093777852477116417, 936194589043683328, 10472...","1093777852477116417, 936194589043683328, 10472...",93.0,116.0,0.0,22.545205,49.264865,60.043243,fake
789,False,2017-12-09 09:10:03+00:00,Wed Feb 24 22:58:57 +0000 2016,False,True,PROUD PATRIOT SUPPORTING TRUMP!! PRO-AMERICAN....,twitter.com/i/web/status/9…,thekind-heartedsmartaleck.blogspot.com,https://twitter.com/i/web/status/9394218766187...,http://thekind-heartedsmartaleck.blogspot.com/,0.0,False,74641.0,False,6903.0,False,4533.0,True,True,9.394219e+17,9.394219e+17,702628810483765249,10228272,7.026288e+17,8.529181e+16,False,False,False,en,86.0,"Orlando, FL",#TRUMP2020,YouTube,False,False,False,F5F8FA,None,None,False,https://pbs.twimg.com/profile_banners/70262881...,https://pbs.twimg.com/profile_images/111902430...,http://pbs.twimg.com/profile_images/1119024306...,1DA1F2,C0DEED,DDEEF6,333333,True,False,0.0,False,PolyPatriot,YouTube,"<a href=""http://twitter.com/download/iphone"" r...",155849.0,Facebook Picks Sides in Alabama Senate Race=&g...,none,True,None,https://t.co/o4Qq2UFraj,http://t.co/PYrnTHpCvI,False,,,120.0,143.0,0.0,22.545205,49.264865,60.043243,fake
790,False,2017-08-29 17:54:38+00:00,Tue Sep 15 02:55:59 +0000 2009,False,True,,houstonchronicle-tv.com/floyd-mayweath…,thekind-heartedsmartaleck.blogspot.com,http://houstonchronicle-tv.com/floyd-mayweathe...,http://thekind-heartedsmartaleck.blogspot.com/,1.0,False,11.0,False,24.0,False,42.0,True,False,9.025903e+17,9.025903e+17,74346965,10228272,7.434696e+07,8.529181e+16,False,Fa

In [ ]:
bigdataPKL.to_pickle('bigdataClean.pkl')

## Hash followers and following

In [27]:
#convert to list
bigdataPKL.following = bigdataPKL.following.apply(lambda x: x[1:-1].split(','))
bigdataPKL.followers = bigdataPKL.followers.apply(lambda x: x[1:-1].split(','))
#Order
try:
    for row in bigdataPKL['following'].keys(): 
        bigdataPKL['following'][row] = bigdataPKL['following'][row].sort(key=int)
    for row in bigdataPKL['followers'].keys():
        bigdataPKL['followers'][row] = bigdataPKL['followers'][row].sort(key=int)
except ValueError:
    pass
#convert to tuple
try:
    for row in bigdataPKL['following'].keys(): 
        bigdataPKL['following'][row] = tuple(bigdataPKL['following'][row])
    for row in bigdataPKL['followers'].keys():
        bigdataPKL['followers'][row] = tuple(bigdataPKL['followers'][row])
except ValueError:
    pass
#hash
try:
    for row in bigdataPKL['following'].keys(): 
        bigdataPKL['following'][row] = hash(bigdataPKL['following'][row])
    for row in bigdataPKL['followers'].keys():
        bigdataPKL['followers'][row] = hash(bigdataPKL['followers'][row])
except ValueError:
    pass

C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.

In [28]:
bigdataPKL.tail()

,contributors_enabled_user,created_at,created_at_user,default_profile_image_user,default_profile_user,description_user,display_url_zero_urls_entities,display_url_zero_urls_url_entities_user,expanded_url_zero_urls_entities,expanded_url_zero_urls_url_entities_user,favorite_count,favorited,favourites_count_user,follow_request_sent_user,followers_count_user,following_user,friends_count_user,geo_enabled_user,has_extended_profile_user,id,id_str,id_str_user,id_str_zero_user_mentions_entities,id_user,id_zero_user_mentions_entities,indices_zero_urls_entities,indices_zero_urls_url_entities_user,indices_zero_user_mentions_entities,is_quote_status,is_translation_enabled_user,is_translator_user,lang,listed_count_user,location_user,name_user,name_zero_user_mentions_entities,notifications_user,possibly_sensitive,possibly_sensitive_appealable,profile_background_color_user,profile_background_image_url_https_user,profile_background_image_url_user,profile_background_tile_user,profile_banner_url_user,profile_image_url_https_user,profile_image_url_user,profile_link_color_user,profile_sidebar_border_color_user,profile_sidebar_fill_color_user,profile_text_color_user,profile_use_background_image_user,protected_user,retweet_count,retweeted,screen_name_user,screen_name_zero_user_mentions_entities,source,statuses_count_user,symbols_entities,text,translator_type_user,truncated,url_user,url_zero_urls_entities,url_zero_urls_url_entities_user,verified_user,followers,following,label
788,False,2017-05-20 14:11:39+00:00,Sun Oct 25 14:01:48 +0000 2009,False,True,,fb.me/1fYsjNaLC,thekind-heartedsmartaleck.blogspot.com,http://fb.me/1fYsjNaLC,http://thekind-heartedsmartaleck.blogspot.com/,0.0,False,16.0,False,47.0,False,87.0,True,True,8.659330e+17,8.659330e+17,85085807,10228272,8.508581e+07,8.529181e+16,"[93, 116]","[0, 23]","[0, 12]",False,False,False,en,7.0,Gassaway WV,Terry Bly,YouTube,False,False,False,C0DEED,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/13552184...,https://pbs.twimg.com/profile_images/249393875...,http://pbs.twimg.com/profile_images/2493938757...,1DA1F2,C0DEED,DDEEF6,333333,True,False,0.0,False,TlbBly,YouTube,"<a href=""http://www.facebook.com/twitter"" rel=...",90486.0,[],WORSE THAN HITLER! Trey Gowdy’s Son Found In A...,none,False,None,https://t.co/dxpndRqHjL,http://t.co/PYrnTHpCvI,False,3016169065007908985,3016169065007908985,fake
789,False,2017-12-09 09:10:03+00:00,Wed Feb 24 22:58:57 +0000 2016,False,True,PROUD PATRIOT SUPPORTING TRUMP!! PRO-AMERICAN....,twitter.com/i/web/status/9…,thekind-heartedsmartaleck.blogspot.com,https://twitter.com/i/web/status/9394218766187...,http://thekind-heartedsmartaleck.blogspot.com/,0.0,False,74641.0,False,6903.0,False,4533.0,True,True,9.394219e+17,9.394219e+17,702628810483765249,10228272,7.026288e+17,8.529181e+16,"[120, 143]","[0, 23]","[0, 12]",False,False,False,en,86.0,"Orlando, FL",#TRUMP2020,YouTube,False,False,False,F5F8FA,None,None,False,https://pbs.twimg.com/profile_banners/70262881...,https://pbs.twimg.com/profile_images/111902430...,http://pbs.twimg.com/profile_images/1119024306...,1DA1F2,C0DEED,DDEEF6,333333,True,False,0.0,False,PolyPatriot,YouTube,"<a href=""http://twitter.com/download/iphone"" r...",155849.0,[],Facebook Picks Sides in Alabama Senate Race=&g...,none,True,None,https://t.co/o4Qq2UFraj,http://t.co/PYrnTHpCvI,False,3430018387555,3430018387555,fake
790,False,2017-08-29 17:54:38+00:00,Tue Sep 15 02:55:59 +0000 2009,False,True,,houstonchronicle-tv.com/floyd-mayweath…,thekind-heartedsmartaleck.blogspot.com,http://houstonchronicle-tv.com/floyd-mayweathe...,http://thekind-heartedsmartaleck.blogspot.com/,1.0,False,11.0,False,24.0,False,42.0,True,False,9.025903e+17,9.025903e+17,74346965,10228272,7.434696e+07,8.529181e+16,"[82, 105]","[0, 23]","[0, 12]",False,False,False,en,0.0,Louisville Ky,Eugene Angton,YouTube,False,False,False,C0DEED,https://abs.twimg.com/images/themes/theme1/bg.png,http://

In [51]:
#try:
#    for row in bigdataPKL['symbols_entities'].keys(): 
#        bigdataPKL['symbols_entities'][row] = bigdataPKL['symbols_entities'][row].strip('][')
#    for row in bigdataPKL['symbols_entities'].keys():
#        bigdataPKL['symbols_entities'][row] = bigdataPKL['symbols_entities'][row].strip('][')
#except ValueError:
#    pass
#bigdataPKL.tail()

#bigdataPKL['symbols_entities'] = bigdataPKL['url_user'].astype(str) 
#bigdataPKL.symbols_entities = bigdataPKL.symbols_entities.apply(lambda x: x[1:-1].split(' '))

#maxLen = 0
#for row in bigdataPKL['symbols_entities'].keys():
#    if len(bigdataPKL['symbols_entities'][row])>maxLen:
#        print(bigdataPKL['symbols_entities'][row])
#        maxLen = len(bigdataPKL['symbols_entities'][row])
#print(maxLen)
bigdataPKL.info()

C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\henri\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


5


In [52]:
#maxLen = 0
#for row in bigdataPKL['symbols_entities'].keys():
#    if len(bigdataPKL['symbols_entities'][row])>maxLen:
#        print(bigdataPKL['symbols_entities'][row])
#        maxLen = len(bigdataPKL['symbols_entities'][row])
#print(maxLen)

['']
["'text':", "'AIG',", "'indices':", '[0,', '4]']
5


In [8]:
#l = [3, 8, 7, 2]
#l = tuple(i for i in l)
#print(hash(l))
#l = [3, 8, 7, 4] 
#l = tuple(i for i in l)
#print(hash(l))
bigdataPKL.to_pickle('bigdataHashed.pkl')

,contributors_enabled_user,created_at,created_at_user,default_profile_image_user,default_profile_user,description_user,display_url_zero_urls_entities,display_url_zero_urls_url_entities_user,expanded_url_zero_urls_entities,expanded_url_zero_urls_url_entities_user,favorite_count,favorited,favourites_count_user,follow_request_sent_user,followers_count_user,following_user,friends_count_user,geo_enabled_user,has_extended_profile_user,id,id_str,id_str_user,id_str_zero_user_mentions_entities,id_user,id_zero_user_mentions_entities,indices_zero_urls_entities,indices_zero_urls_url_entities_user,indices_zero_user_mentions_entities,is_quote_status,is_translation_enabled_user,is_translator_user,lang,listed_count_user,location_user,name_user,name_zero_user_mentions_entities,notifications_user,possibly_sensitive,possibly_sensitive_appealable,profile_background_color_user,profile_background_image_url_https_user,profile_background_image_url_user,profile_background_tile_user,profile_banner_url_user,profile_image_url_https_user,profile_image_url_user,profile_link_color_user,profile_sidebar_border_color_user,profile_sidebar_fill_color_user,profile_text_color_user,profile_use_background_image_user,protected_user,retweet_count,retweeted,screen_name_user,screen_name_zero_user_mentions_entities,source,statuses_count_user,symbols_entities,text,translator_type_user,truncated,url_user,url_zero_urls_entities,url_zero_urls_url_entities_user,verified_user,followers,following,label
0,False,2015-10-09 21:18:53+00:00,Fri Jun 27 23:15:54 +0000 2014,False,False,Sagittarius♐️ | @darrencriss🕶 | Demi Lovato💖 |...,eventbrite.com/e/derry-town-h…,usdemocrats.com,https://www.eventbrite.com/e/derry-town-hall-m...,None,1.0,False,3936.0,False,632.0,False,950.0,True,True,6.525941e+17,6.525941e+17,2592090978,15745368,2.592091e+09,1.574537e+07,"[117, 140]","[0, 23]","[7, 18]",False,False,False,en,14.0,"Jericho, NY",Paige Johnson,Marco Rubio,False,False,False,2F0EEB,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_banners/25920909...,https://pbs.twimg.com/profile_images/112189326...,http://pbs.twimg.com/profile_images/1121893261...,ED0E0E,FFFFFF,DDEEF6,333333,True,False,2.0,False,Paige_Johnson17,marcorubio,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",3109.0,[],Missed @marcorubio in NH this week? Come to a ...,none,False,None,https://t.co/FlrKPldQ76,http://t.co/DFfnwVB1mD,False,3430018387555,3430018387555,real
1,False,2013-03-22 06:36:15+00:00,Tue Nov 10 15:44:34 +0000 2009,False,False,Latest bill activity in the Michigan Legislature,1.usa.gov/15nzBw1,legislature.mi.gov,http://1.usa.gov/15nzBw1,http://www.legislature.mi.gov,0.0,False,1.0,False,1370.0,False,0.0,False,False,3.149889e+17,3.149889e+17,88953774,8953122,8.895377e+07,8.529181e+16,"[91, 113]","[0, 22]","[0, 11]",False,False,False,en,68.0,"Lansing, Michigan",Michigan Legislature,PolitiFact,False,False,False,FFFFFF,https://abs.twimg.com/images/themes/theme1/bg.png,http://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/18188773...,https://pbs.twimg.com/profile_images/519651617...,http://pbs.twimg.com/profile_images/519651617/...,0084B4,C0DEED,E5D7C8,333333,True,False,0.0,False,michiganleg,PolitiFact,"<a href=""http://www.legislature.mi.gov"" rel=""n...",84579.0,[],2013-HB-4469 Public employees and officers; et...,none,False,http://t.co/7t82uxHYgJ,http://t.co/8qY8mG0LMT,http://t.co/7t82uxHYgJ,False,3430018387555,3430018387555,real
2,False,2008-03-03 06:10:37+00:00,Wed Mar 21 16:48:44 +0000 2007,False,False,Breaking news and features from the National D...,twitter.com/i/web/status/1…,nytimes.com/national,http://aol.it/ZqFuvl,http://nytimes.com/national,0.0,False,2010.0,False,79814.0,False,820.0,True,False,7.659183e+08,7.659183e+08,1767741,8953122,1.767741e+06,8.529181e+16,"[117, 140]","[0, 23]","[0, 11]",False,False,False,en,2882.0,From Sea to Shining Sea,NYT National News,PolitiFact,False,